In [16]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

In [55]:
df = pd.read_csv('Ulst6Jan87.csv')
df.head()
df['geometry'] = list(zip(df['LONG'], df['LAT'])) 
df['geometry'] = df['geometry'].apply(Point) 
HistData = gpd.GeoDataFrame(df)
gdf = gdf.set_crs("EPSG:4326")
#gdf.to_file('HistoricGravityPoints.shp') # I wish to create a shapefile for future use

In [56]:
HistData.head() #sanity check

,id,LAT (DDMM.mm),LONG (DDMM.mm),LATD,LATM,LATH,LOND,LONM,LONH,LAT,...,BA,BA (mgal),CODE,LAT (mgal),FAC,Gfa,BC,Gb,Gbt,geometry
0,U1 1A02,551776,-61241,55,17,76,-6,12,41,55.2960,...,-700,-7.00,NaN,981531.72,29.63,2.52,10.87,-8.35,-7.01,POINT (-6.20680 55.29600)
1,U1 1A03,551773,-61424,55,17,73,-6,14,24,55.2955,...,-658,-6.58,NaN,981531.68,25.95,1.66,9.52,-7.86,-6.59,POINT (-6.23730 55.29550)
2,U1 1A04,551798,-61523,55,17,98,-6,15,23,55.2997,...,-387,-3.87,NaN,981532.03,27.37,5.44,10.04,-4.60,-3.86,POINT (-6.25380 55.29970)
3,U1 1A05,551756,-6162,55,17,56,-6,16,2,55.2927,...,-584,-5.84,NaN,981531.44,0.12,-6.68,0.05,-6.72,-5.83,POINT (-6.26700 55.29270)
4,U1 1B02,551573,-61119,55,15,73,-6,11,19,55.2622,...,-245,-2.45,NaN,981528.85,0.43,-2.53,0.16,-2.69,-2.44,POINT (-6.18650 55.26220)


In [ ]:
gdf

In [24]:
df = pd.read_csv('OSNIAbsoluteGravity.csv')

In [ ]:
df.head()

In [26]:
del df['Easting.1'], df['Northing.1'], df['Unnamed: 8'], df['latitude'], df ['longitude'] #These attributions are confusing

In [27]:
df.head() yet another sanity check

,Unnamed: 0,Station Name,GNSS Observation Date,Easting,Northing,Elevation,Altitude
0,1,Cushendall,27/02/2020,724301.9337,927608.7551,1.100000,57.076594
1,2,Bangor,09/03/2020,750484.3709,882233.5231,4.671654,60.425980
2,3,Belfast,24/08/2022,733995.2420,876707.6150,15.452000,71.575989
3,3,Belfast IGSN71,24/08/2022,736473.4086,876421.7018,1.760000,57.839865
4,4,Moneymore,11/03/2020,685772.5538,883608.0075,53.296145,110.195640


In [36]:
df['geometry'] = list(zip(df['Easting'], df['Northing'])) 
df['geometry'] = df['geometry'].apply(Point) 
gdf = gpd.GeoDataFrame(df)
gdf = gdf.set_crs("EPSG:2157") #This was a later addition, I was slow to realise I needed to set a CRS at this point before using gdf.to_crs.

In [37]:
gdf.crs #Checking current CRS

<Projected CRS: EPSG:2157>
Name: IRENET95 / Irish Transverse Mercator
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Ireland - onshore. United Kingdom (UK) - Northern Ireland (Ulster) - onshore.
- bounds: (-10.56, 51.39, -5.34, 55.43)
Coordinate Operation:
- name: Irish Transverse Mercator
- method: Transverse Mercator
Datum: IRENET95
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [38]:
gdf = gdf.to_crs("EPSG:4326") #The source data geometry is EPSG 2157 (Irish Transverse Mercator)
#gdf.to_file('AbsoluteGravityPoints.shp') #As with the first dataset, I wish to create a shapefile

In [39]:
gdf.crs # Sanity check after changing CRS

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [40]:
gdf.head() Another sanity check. 

,Unnamed: 0,Station Name,GNSS Observation Date,Easting,Northing,Elevation,Altitude,geometry
0,1,Cushendall,27/02/2020,724301.9337,927608.7551,1.100000,57.076594,POINT (-6.05323 55.08034)
1,2,Bangor,09/03/2020,750484.3709,882233.5231,4.671654,60.425980,POINT (-5.66718 54.66576)
2,3,Belfast,24/08/2022,733995.2420,876707.6150,15.452000,71.575989,POINT (-5.92513 54.62080)
3,3,Belfast IGSN71,24/08/2022,736473.4086,876421.7018,1.760000,57.839865,POINT (-5.88692 54.61757)
4,4,Moneymore,11/03/2020,685772.5538,883608.0075,53.296145,110.195640,POINT (-6.66953 54.69327)
